### 1. Importing all the packages

In [2]:
import os
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns  
from pandas import set_option
plt.style.use('ggplot')
from scipy.stats import randint
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV  
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
import warnings
import numpy as np
warnings.filterwarnings("ignore", category=FutureWarning)


### 2. Reading the Credit Deafult data

In [3]:
os.chdir("C:/D Drive Data/Max_Life/Data/")
data=pd.read_excel("case_study_data.xlsx")
data.head()

,checkin_acc,duration,credit_history,purpose,amount,svaing_acc,present_emp_since,inst_rate,personal_status,other_debtors,...,property,age,inst_plans,housing,num_credits,job,dependents,telephone,foreign_worker,status
0,A11,9,A34,A43,1754,A65,A75,6,A93,A101,...,A121,101,A143,A152,3,A173,2,A192,A201,1
1,A12,72,A32,A43,8927,A61,A73,3,A92,A101,...,A121,33,A143,A152,2,A173,2,A191,A201,2
2,A14,18,A34,A46,3144,A61,A74,3,A93,A101,...,A121,74,A143,A152,2,A172,3,A191,A201,1
3,A11,63,A32,A42,11823,A61,A74,3,A93,A103,...,A122,68,A143,A153,2,A173,3,A191,A201,1
4,A11,36,A33,A40,7305,A61,A73,5,A93,A101,...,A124,80,A143,A153,3,A173,3,A191,A201,2


### 3. Data Exploration

In [4]:
#Replacing 1 by 0 and 2 by 1
data.status.replace([1,2], [0,1], inplace=True)

In [5]:
##subsetting data for calculating woe values:
data_woe=data

In [6]:
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        }) 
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    dset = dset.sort_values(by='WoE')
    return dset, iv

In [7]:
lst1= []
IV_df = pd.DataFrame(columns=['Variable','IV'])
for col in data_woe.columns:
    if col == 'status': continue
    else:
          df, iv= calculate_woe_iv(data_woe, col, 'status')
    lst1.append(df)
    IV_df = IV_df.append({
                "Variable" :col ,
                "IV" : iv,
                },ignore_index=True)

C:\Users\Shikha.roy\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
lst1[0]

,Value,All,Good,Bad,Distr_Good,Distr_Bad,WoE,IV
0,A11,274,139,135,0.198571,0.450000,-0.818099,0.205693
1,A12,269,164,105,0.234286,0.350000,-0.401392,0.046447
3,A13,63,49,14,0.070000,0.046667,0.405465,0.009461
2,A14,394,348,46,0.497143,0.153333,1.176263,0.404410


In [9]:
data_checkin_acc= lst1[0]
data_checkin_acc = data_checkin_acc.rename(columns={"Value":"checkin_acc","WoE":"WoEcheckin_acc"})


In [10]:
data= pd.merge(data, data_checkin_acc[['checkin_acc','WoEcheckin_acc']], on='checkin_acc')

In [11]:
data_duration= lst1[1]
data_duration = data_duration.rename(columns={"Value":"duration","WoE":"WoEduration"})


In [12]:
data_credit_history= lst1[2]
data_credit_history = data_credit_history.rename(columns={"Value":"credit_history","WoE":"WoEcredit_history"})


In [13]:
data_age= lst1[12]
data_age = data_age.rename(columns={"Value":"age","WoE":"WoEage"})


In [14]:
data= pd.merge(data, data_duration[['duration','WoEduration']], on='duration')

In [15]:
data= pd.merge(data, data_credit_history[['credit_history','WoEcredit_history']], on='credit_history')

In [16]:
data= pd.merge(data, data_age[['age','WoEage']], on='age')

In [17]:
data=data.drop(['checkin_acc','duration','credit_history','age'],axis=1)

In [18]:
#Capping outlier if any in the amount column

In [19]:
data['amount'] = np.where(data['amount'] <data['amount'].quantile(.05), data['amount'].quantile(.05),data['amount'])

In [20]:
data['amount']  = np.where(data['amount'] >data['amount'].quantile(.95), data['amount'].quantile(.95),data['amount']) 

In [21]:
# perform one hot encoding with k - 1, it automatically drop the first.
#not ordinal variables
one_hot_enc=pd.get_dummies(data[['purpose','svaing_acc','present_emp_since','personal_status','other_debtors',  'property', 'inst_plans', 'housing','job', 'telephone', 'foreign_worker']])

In [22]:
data=pd.concat([data,one_hot_enc],axis=1)

In [23]:
data.head()

,purpose,amount,svaing_acc,present_emp_since,inst_rate,personal_status,other_debtors,residing_since,property,inst_plans,...,housing_A152,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreign_worker_A201,foreign_worker_A202
0,A43,1754.0,A65,A75,6,A93,A101,6,A121,A143,...,1,0,0,0,1,0,0,1,1,0
1,A45,5808.0,A61,A71,3,A92,A101,6,A123,A143,...,1,0,0,0,1,0,0,1,1,0
2,A46,1799.0,A61,A74,6,A92,A101,6,A122,A143,...,1,0,0,0,0,1,0,1,1,0
3,A40,1290.0,A61,A75,2,A92,A101,6,A124,A143,...,1,0,0,0,1,0,0,1,1,0
4,A40,1063.9,A64,A74,5,A92,A101,6,A121,A143,...,1,0,0,1,0,0,0,1,1,0


In [24]:
data1=data

In [25]:
data=data.drop(['purpose','svaing_acc','present_emp_since','personal_status','other_debtors',  'property', 'inst_plans', 'housing','job', 'telephone', 'foreign_worker'],axis=1)

In [26]:
varlist=data.columns.difference(['status'])

In [27]:
data[varlist]=data[varlist].apply(lambda x: (x-min(x))/(max(x)-min(x)))

### Oversampling

In [28]:
from sklearn.utils import resample
status_majority = data[data.status == 0]
status_minority = data[data.status == 1]
status_minority_oversampled = resample(status_minority, replace = True, n_samples =  700, random_state = 333)
data_oversampled = pd.concat([status_majority, status_minority_oversampled])

In [29]:
data_oversampled.status.value_counts()

1    700
0    700
Name: status, dtype: int64

In [30]:
train,val=train_test_split(data_oversampled,test_size=0.3)

In [31]:
train.shape

(980, 55)

In [32]:
#predictors and dependent for train data
X=train[varlist]
Y=train.iloc[:,5]

In [33]:
#predictors and dependent for validation data
X_val=val[varlist]
Y_val=val.iloc[:,5]

In [38]:
1/300

0.0033333333333333335

In [39]:
1/700

0.0014285714285714286

In [54]:
Log_Pipeline = Pipeline([( 'LogReg', LogisticRegression(random_state=123)) ])

parameters = dict({ 'LogReg__penalty': ['l1','l2','elasticnet'],'LogReg__C': [0.01,0.001,0.0001,0.02,0.03,.04,.05,.1,.2], 'LogReg__max_iter': [100,500,1000],'LogReg__tol': [ 0.0001,.001,0.01 ], 'LogReg__solver': ['liblinear','saga'],'LogReg__l1_ratio' :[0,0.5,1]})

Grid_log = GridSearchCV(Log_Pipeline, parameters, cv=10, scoring='roc_auc')

In [55]:
log_fit=Grid_log.fit(X,Y)

C:\Users\Shikha.roy\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
C:\Users\Shikha.roy\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
C:\Users\Shikha.roy\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
C:\Users\Shikha.roy\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
C:\Users\Shikha.roy\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1501: UserWarning: l1_ratio parameter is o

In [56]:
best_log = log_fit.best_estimator_

In [62]:
best_log.score(X,Y)

0.7510204081632653

In [63]:
best_log

Pipeline(memory=None,
         steps=[('LogReg',
                 LogisticRegression(C=0.2, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=0, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=123,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [64]:
import pickle
filename = 'Model_best_logistic.sav'
pickle.dump(best_log, open(filename, 'wb'))
 
# load the model from disk
log_model_final = pickle.load(open(filename, 'rb'))
score1 = log_model_final.score(X_val, Y_val)
print(score1)

0.7619047619047619


In [36]:
# load the model from disk

import pickle
filename = 'Model_best_logistic.sav'
log_model_final = pickle.load(open(filename, 'rb'))

In [42]:
# Predcited probability of each class.
y_pred_prob = log_model_final.predict_proba(X_val)
# Predicted value of each class
y_pred_class = log_model_final.predict(X_val)

print(metrics.classification_report(Y_val, y_pred_class))

              precision    recall  f1-score   support

           0       0.90      0.71      0.79       224
           1       0.47      0.76      0.58        76

    accuracy                           0.72       300
   macro avg       0.68      0.74      0.69       300
weighted avg       0.79      0.72      0.74       300



In [43]:
cMatrix = confusion_matrix(Y_val, y_pred_class)
print(cMatrix)
print("Customers that were falsely classified as Defaulter =  %.1f Percent" %(cMatrix[0][1]/ sum(sum(cMatrix))*100 ) )
print("Customers that were falsely classified as Non-Defaulter = %.1f Percent"%(cMatrix[1][0]/ sum(sum(cMatrix))*100 ) )
print("Customers that were truly classified as Defaulter = %.1f Percent"% (cMatrix[1][1]/ sum(sum(cMatrix))*100 ) )
print("Customers that were truly classified as Non-Defaulter= %.1f Percent"% (cMatrix[0][0]/ sum(sum(cMatrix))*100 ))


[[159  65]
 [ 18  58]]
Customers that were falsely classified as Defaulter =  21.7 Percent
Customers that were falsely classified as Non-Defaulter = 6.0 Percent
Customers that were truly classified as Defaulter = 19.3 Percent
Customers that were truly classified as Non-Defaulter= 53.0 Percent


In [46]:
from sklearn.inspection import permutation_importance
r = permutation_importance(log_model_final, X, Y,n_repeats=5,random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{data.columns[i]:<8}"f"{r.importances_mean[i]:.3f}"f" +/- {r.importances_std[i]:.3f}")

inst_rate0.066 +/- 0.007
purpose_A430.020 +/- 0.005
residing_since0.017 +/- 0.006
present_emp_since_A710.011 +/- 0.005
num_credits0.011 +/- 0.003
job_A1710.010 +/- 0.005
svaing_acc_A640.006 +/- 0.002
inst_plans_A1430.004 +/- 0.001
purpose_A440.003 +/- 0.001
property_A1230.003 +/- 0.001
purpose_A4100.002 +/- 0.001


In [47]:
from sklearn.inspection import permutation_importance
r = permutation_importance(log_model_final, X_val, Y_val,n_repeats=5,random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{data.columns[i]:<8}"f"{r.importances_mean[i]:.3f}"f" +/- {r.importances_std[i]:.3f}")

inst_rate0.056 +/- 0.015
num_credits0.031 +/- 0.013
other_debtors_A1030.020 +/- 0.004
dependents0.019 +/- 0.007
purpose_A460.015 +/- 0.006
amount  0.013 +/- 0.004
other_debtors_A1020.010 +/- 0.005
svaing_acc_A650.008 +/- 0.003
housing_A1520.007 +/- 0.002
purpose_A450.006 +/- 0.002


In [50]:
#Printing the log-loss for both trian and test data
from sklearn.metrics import log_loss
train_loss = log_loss(Y, log_model_final.predict_proba(X)[:,1])
test_loss  =log_loss(Y_val, y_pred_prob)
print("Log_loss on train data is :{}".format(train_loss))
print("Log_loss on test data is :{}".format(test_loss))

Log_loss on train data is :0.49065821231734924
Log_loss on test data is :0.5473316887567744
